<a href="https://colab.research.google.com/github/RS07GIT/wordle_solver/blob/main/Main.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Import PyDrive and associated libraries.
# This only needs to be done once per notebook.
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials
import string

# Authenticate and create the PyDrive client.
# This only needs to be done once per notebook.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

# List .txt files in the root.
#
# Search query reference:
# https://developers.google.com/drive/v2/web/search-parameters
file_id = '1YW9tCtl1w_a8lnYttkvH2afvPo2mWGIc'
downloaded = drive.CreateFile({'id': file_id})
WordList = downloaded.GetContentString().split('\n')

In [ ]:
import string
import random
from functools import lru_cache


letters = list(string.ascii_lowercase[0:26])

@lru_cache(maxsize=None)
def score_letter(letter):
  score = 0
  for word in WordList:
    if letter in word:
      score += 1
  return score

@lru_cache(maxsize=None)
def score_word(word):
  score = 0
  scoredLetters = ''
  for letter in word:
    if not letter in scoredLetters:
      score += score_letter(letter)
      scoredLetters += letter
  return score

def letter_tried_check(letter, position, tried_letters):
  for tried_letter in tried_letters:
    if tried_letter[0] == letter:
      return tried_letter[0] == position
  return True, False

def remove_letters_from_word_position(word, positions):
  new_word = ''

  for index, letter in enumerate(word):
    if index not in positions:
      new_word += letter
  return new_word

def remove_letters_from_word_letters(word, letters):
  new_word = ''
  
  for letter in word:
    if letter not in letters:
      new_word += letter
      
  return new_word

def word_check_confirmed(word, letter_dic):
  location_list = []

  for k, v in letter_dic.items():
    if v[0] and v[1] == 99:
      for position in v[2]:
        if word[position] == k:
          return False, location_list

  for k, v in letter_dic.items():
    if v[0] and v[1] < 5:
      if word[v[1]] == k:
        location_list.append(v[1])
      else:
        return False, location_list
  return True, location_list

def word_check_unconfirmed(word, letter_dic):
  letter_list = []
      
  for k, v in letter_dic.items():
    if v[0] and v[1] == 99:
      if k in word:
        letter_list.append(k)
      else:
        return False, letter_list
  return True, letter_list

def word_check_bad(word, letter_dic):
  for k, v in letter_dic.items():
    if v[0] == False:
      if k in word:
        return False
  return True

def word_guess(letter_dic):
  word_dic = {}

  for word in WordList:
    result, location_list  = word_check_confirmed(word, letter_dic)

    if result:
      updated_word = remove_letters_from_word_position(word, location_list)
      result, letter_list = word_check_unconfirmed(updated_word, letter_dic)

      if result:
        updated_word = remove_letters_from_word_letters(updated_word, letter_list)
        result = word_check_bad(updated_word, letter_dic)
      
        if result:
          word_dic[word] = score_word(word)
  
  sorted_word_dic = dict(sorted(word_dic.items(), key=lambda x: x[1], reverse=True))
  if bool(sorted_word_dic):
    return next(iter(sorted_word_dic))
  else:
    return 'fucked'

def update_tried_letters(tried_letters, new_guess, target_word):
  for i in range(5):
    if new_guess[i] in target_word:
      triedColumnArray = []
      if new_guess[i] in tried_letters:
        v = tried_letters[new_guess[i]]
        v[2].append(i)
        triedColumnArray = v[2]
      else:
        triedColumnArray = [i]

      if new_guess[i] == target_word[i]:
        tried_letters[new_guess[i]] = [True, i, triedColumnArray]
      else:
        tried_letters[new_guess[i]] = [True, 99, triedColumnArray]
    else:
      tried_letters[new_guess[i]] = ([False, 99, [i]])
  return tried_letters

def run():
  target_word = 'abuse'
  guessed_correctly = 0
  guess_tries = 0

  for word in WordList:
    guessed_word = ''
    tried_letters = {}
    guessTries = 0

    while guessed_word != word and guessed_word != 'fucked':
      guessTries += 1
      guessed_word = word_guess(tried_letters)

      if guessed_word != 'fucked':
        tried_letters = update_tried_letters(tried_letters, guessed_word, word)
    else:
      if guessed_word != 'fucked':
        guessed_correctly += 1
        guess_tries += guessTries
    
  print('% Correct:', (guessed_correctly) * 100)          
  print('Average tries:', guess_tries)

def inputToTriedLetters(tried_letters, player_input, guessed_word):
  for i in range(5):
    if player_input[i] != '0':
      triedColumnArray = []
      if guessed_word[i] in tried_letters:
        v = tried_letters[guessed_word[i]]
        v[2].append(i)
        triedColumnArray = v[2]
      else:
        triedColumnArray = [i]

      if player_input[i] == '2':
        tried_letters[guessed_word[i]] = [True, i, triedColumnArray]
      else:
        tried_letters[guessed_word[i]] = [True, 99, triedColumnArray]
    else:
      tried_letters[guessed_word[i]] = ([False, 99, [i]])
  return tried_letters


def new_game():
  tried_letters = {}

  for i in range(6):
    guess = word_guess(tried_letters)
    print(guess)
    results = input()

    if results == '1':
      break
    else:
      tried_letters = inputToTriedLetters(tried_letters, results, guess)

new_game()

alert
00111
tenor
11012
outer
20122
other
1
